In [1]:
import scipy.stats
from collections import defaultdict
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import re


In [2]:
import re
# Function to parse the string and remove unwanted characters
def parse_string(text_string):
    """"
        Parse the input string and tokenize it using regular expressisons:
        First clean the string such that it does not have any punctuation or number, it must only have a-z and A-Z.
        Please note that while doing this, the spaces much not get disturbed, but in case of multiple spaces convert 
        them to one space.
        Then convert the string to lower case and return its words as a list of strings.
        
        Example:
        --------
        Input :  computer scien_tist-s are,,,  the  rock__stars of tomorrow_ <cool>  ????
        Output:  ['computer', 'scientists', 'are', 'the', 'rockstars', 'of', 'tomorrow']
        
        Parameters:
        ----------
        string: string to be parsed...
        re: regular expression to be used for the tokenization.
        
        Returns:
        ---------
        list of tokens extracted from the string...
    """ 
    # Check if the input is bytes, and decode it to string if necessary
    if isinstance(text_string, bytes):
        text_string = text_string.decode('utf-8')
    regex_pat = r'[^\w\s]'
    # Remove HTML tags and text inside parentheses
    new_string = re.sub(r'<[^>]*>|\([^)]*\)', '', text_string)
    # Remove non-alphanumeric characters
    new_string = re.sub(regex_pat, '', new_string)
    regex_pat2 = r'-+'
    # Replace multiple hyphens with a single space
    new_string = re.sub(regex_pat2, ' ', new_string)
    return new_string


In [3]:

def parse_string1(text_string): 
    """"
        Parse the input string and tokenize it using regular expressisons:
        First clean the string such that it does not have any punctuation or number, it must only have a-z and A-Z.
        Please note that while doing this, the spaces much not get disturbed, but in case of multiple spaces convert 
        them to one space.
        Then convert the string to lower case and return its words as a list of strings.
        
        Example:
        --------
        Input :  computer scien_tist-s are,,,  the  rock__stars of tomorrow_ <cool>  ????
        Output:  ['computer', 'scientists', 'are', 'the', 'rockstars', 'of', 'tomorrow']
        
        Parameters:
        ----------
        string: string to be parsed...
        re: regular expression to be used for the tokenization.
        
        Returns:
        ---------
        list of tokens extracted from the string...
    """

    # Check if the input is bytes, and decode it to string if necessary
    if isinstance(text_string, bytes):
        text_string = text_string.decode('utf-8')
    regex_pat = r'[^\w\s]'
    new_string = re.sub(r'<[^>]*>|\([^)]*\)', '', text_string)
    new_string = re.sub(regex_pat,'',new_string)
    regex_pat2 = r'-+'
    new_string = re.sub(regex_pat2,'',new_string)
    # new_string = re.sub(r'<[^>]*>|\([^)]*\)', '', new_string)
    word_tokens = re.findall(r'\w+', new_string)
    reg_pat3 = r'[^a-zA-Z]'
    new_clean = []
    for word in word_tokens:
        temp = re.sub(reg_pat3, '', word)
        new_clean.append(temp)
    return new_clean

clean = parse_string1('''        computer scien_tist-s are,,,  the 
                    # rock__stars of tomorrow_ <cool>  ????''')
clean

['computer', 'scientists', 'are', 'the', 'rockstars', 'of', 'tomorrow']

In [4]:
def parse_file(filename): # Parse a given file
    """"
        Parameters:
        ----------
        filename: name of text file to be read
   
        
        Returns:
        ---------
        read file as raw string (with \n, \t, \r, etc included)
    """
    lines = []
    with open(filename, 'r') as file:
        lines = file.read()
        #for line in file:
        #    lines.append(line)
    return lines
lines = parse_file('./neg/cv000_29416.txt')
lines

'plot : two teen couples go to a church party , drink and then drive . \nthey get into an accident . \none of the guys dies , but his girlfriend continues to see him in her life , and has nightmares . \nwhat\'s the deal ? \nwatch the movie and " sorta " find out . . . \ncritique : a mind-fuck movie for the teen generation that touches on a very cool idea , but presents it in a very bad package . \nwhich is what makes this review an even harder one to write , since i generally applaud films which attempt to break the mold , mess with your head and such ( lost highway & memento ) , but there are good and bad ways of making all types of films , and these folks just didn\'t snag this one correctly . \nthey seem to have taken this pretty neat concept , but executed it terribly . \nso what are the problems with the movie ? \nwell , its main problem is that it\'s simply too jumbled . \nit starts off " normal " but then downshifts into this " fantasy " world in which you , as an audience membe

In [5]:
def files_to_strings(X):   
    """
        Read an array (or list) of files where each file content is read in a string...
        Input:
        -------
        X an array (or list) of file names
        
        Returns:
        --------
        X as a numpy array with each row containing a read string from the file...
    """
    all_file_data = []
    for file in X:
        current_file_data = []
        with open(file, 'r') as file_content:
            current_file_data = file_content.read() 
        all_file_data.append(current_file_data)
    return np.array(all_file_data)

In [6]:
X = np.array(["./neg/cv000_29416.txt", "./pos/cv000_29590.txt"])
content = files_to_strings(X)

In [7]:
content[0]

'plot : two teen couples go to a church party , drink and then drive . \nthey get into an accident . \none of the guys dies , but his girlfriend continues to see him in her life , and has nightmares . \nwhat\'s the deal ? \nwatch the movie and " sorta " find out . . . \ncritique : a mind-fuck movie for the teen generation that touches on a very cool idea , but presents it in a very bad package . \nwhich is what makes this review an even harder one to write , since i generally applaud films which attempt to break the mold , mess with your head and such ( lost highway & memento ) , but there are good and bad ways of making all types of films , and these folks just didn\'t snag this one correctly . \nthey seem to have taken this pretty neat concept , but executed it terribly . \nso what are the problems with the movie ? \nwell , its main problem is that it\'s simply too jumbled . \nit starts off " normal " but then downshifts into this " fantasy " world in which you , as an audience membe

In [8]:
if content[0]==lines:
    print('Same')

Same


In [9]:
file2 = parse_file("./pos/cv000_29590.txt")
if content[1] == file2:
    print('Same')

Same


In [10]:
# Removing stopwords
def remove_stopwords(tokens):
    new_tokens=[]
    for t in tokens:
        if t not in stop_words and t!='':
            new_tokens.append(t)
            
    return new_tokens

In [11]:
# let's read the stopwords wali file and then make an array of stopwords
file1 = './english.stop'
stop_words = []
with open(file1,'r') as fil:
    for words in fil:
        stop_words.append(words)
stop_words = [word.strip() for word in stop_words]

#### Test Case for the above code

In [12]:
from nose.tools import assert_equal, assert_list_equal
import numpy as np
assert_list_equal(parse_string1("computer scien_tist-s are,,,  the  rock__stars of tomorrow_ <cool>  ????"),
        [u'computer', u'scientists', u'are', u'the', u'rockstars', u'of', u'tomorrow'], "Incorrect cleanning")


strings = files_to_strings(np.array(["./neg/cv000_29416.txt", "./pos/cv000_29590.txt"]))
with open("./neg/cv000_29416.txt") as f:
    text = f.read()

assert_equal(strings[0], text, "At first index should be text of first file")
assert_equal(strings.shape, (2,), "Shape must be (2,) for two files in list")

#### Preparing Test and Training Data

In [13]:
import pandas as pd
import tools as t

tools.py is being imported


In [14]:
import os
posfiles = [os.path.join('./pos', file) for file in os.listdir('./pos')]
negfiles = [os.path.join('./neg', file) for file in os.listdir('./neg')]

In [15]:
import numpy as np
plabels=['pos']*(len(posfiles))
nlabels=['neg']*(len(posfiles))
labels=np.concatenate((plabels,nlabels)) # concatenate the +ve and -ve labels
tX=np.concatenate((posfiles,negfiles))
print ("Training data Dimensions =", tX.shape," Training labels dimensions=", labels.shape)

Training data Dimensions = (2000,)  Training labels dimensions= (2000,)


In [16]:
X=files_to_strings(tX) # read files and convert each file into set of strings and return an numpy array
X = X.reshape((X.shape[0], 1))
#Split the data into two halves training and test set...
#traindata,trainlabels,testdata,testlabels=t.split_data(X,labels)
indices = np.random.permutation(len(X))
split_point = int(len(X) * 0.8)  # 80% training, 20% test
train_idx, test_idx = indices[:split_point], indices[split_point:]
traindata, testdata = X[train_idx], X[test_idx]
trainlabels, testlabels = labels[train_idx], labels[test_idx]
#Find the classes to train
classes=np.unique(labels)

In [17]:
text = traindata[0][0]

#### Before getting into Naive Bayes we will tokenize and preprocess the data

In [18]:
# Working on training data
train_data_tokens = []
for content in traindata:
# for file in traindata:
    # for this current file extract iska content
    #content = parse_file(file)
    # now that we have the content 
    # and as content is array of lines
    # for each line we will extract tokens 
    tokens = []
    for line in content:
        temp_tokens = parse_string(line)
        for ts in temp_tokens:
            tokens.append(ts)
    new_tokens = remove_stopwords(tokens)
    # file 1 k new tokens are on first index
    for t in new_tokens:
        train_data_tokens.append(t)

In [19]:
len(train_data_tokens)

1184066

In [20]:
# Working on test data
test_data_tokens = []
for content in testdata:
# for file in traindata:
    # for this current file extract iska content
    #content = parse_file(file)
    # now that we have the content 
    # and as content is array of lines
    # for each line we will extract tokens 
    tokens = []
    for line in content:
        temp_tokens = parse_string(line)
        for ts in temp_tokens:
            tokens.append(ts)
    new_tokens = remove_stopwords(tokens)
    # file 1 k new tokens are on first index
    for t in new_tokens:
        test_data_tokens.append(t)

In [21]:
len(test_data_tokens)

299834

In [22]:
def get_tokens(text):
    tokens = []
    temp_tokens = []
    tok = parse_string1(text)
    for t in tok:
        temp_tokens.append(t)
    new_tokens = remove_stopwords(temp_tokens)
    return new_tokens

In [23]:
token = get_tokens(text)

In [24]:
len(token)

139

In [25]:
token

['susan',
 'grangers',
 'review',
 'session',
 'bargained',
 'bostonbased',
 'hazmat',
 'elimination',
 'run',
 'scottish',
 'actor',
 'peter',
 'mullan',
 'trusty',
 'assistant',
 'david',
 'caruso',
 'assures',
 'town',
 'engineer',
 'remove',
 'insidious',
 'asbestos',
 'fibers',
 'victorian',
 'hospital',
 'facility',
 'week',
 'erected',
 'deserted',
 'decomposing',
 'danvers',
 'mental',
 'hospital',
 'malevolent',
 'locations',
 'chosen',
 'film',
 'structure',
 'massive',
 'labyrinth',
 'rubblestrewn',
 'corridors',
 'collapsing',
 'floors',
 'stagnant',
 'pools',
 'water',
 'isolation',
 'cells',
 'ominous',
 'surgical',
 'chambers',
 'experimental',
 'prefrontal',
 'lobotomies',
 'performed',
 'task',
 'impossible',
 'time',
 'frame',
 'member',
 'inexperienced',
 'crew',
 'coping',
 'personal',
 'demons',
 'minds',
 'affected',
 'grim',
 'areas',
 'theyre',
 'working',
 'films',
 'title',
 'derived',
 'salvaged',
 'reeltoreel',
 'audiorecorded',
 'sessions',
 'involving',
 '

### Implementing Naiva Bayes Classifier

In [26]:
import nltk
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize

# Download resources if not already installed
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /Users/urwah/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/urwah/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [27]:
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

In [28]:
# YOUR CODE HERE
# raise NotImplementedError()
        
## Hint, you can use python dictionary or default dict for counting the words
# or counter class from collections 

#TODO Complete this class for running the complete classifier... 

#You might need to define auxiliary classes for the complete algorithm..

class NaiveBayes:
    ''' Implements the Naive Bayes For Text Classification... '''

    
    def __init__(self, classes):
        self.vocabulary = set() #this will store all the words (tokens) in the documents
        # this will store the number of words coming under posivitve class and negative class
        #Tracks how many times a specific word appears in each class.
        # example
        # positive: {'smile':1 , 'happy':2}
        self.wordcount_per_class = defaultdict(lambda: defaultdict(int))
        #class count will sotre total number of words coming the domain of +/- classes
        self.class_count = defaultdict(int)
        # this will store the prior probability of each class
        self.prior_probs = defaultdict(int)
        self.doc_total = 0  # Total number of documents

        
        
    def addExample(self, x, y):
        '''
            Add example to corresponding class model ...
            Input
            ---------
            x: example (list of words)
            y: label...
        '''
        # YOUR CODE HERE
        # Update total number of documents
        self.doc_total += 1
        # Update class prior probability (document count for the class)
        self.prior_probs[y] = (self.prior_probs[y] * (self.doc_total - 1) + 1) / self.doc_total
        tokens = get_tokens(x)
        # Update total word count for the given class
        self.class_count[y] += len(tokens)
        # Update word counts and vocabulary
        self.vocabulary.update(tokens)
        for word in tokens:
            self.wordcount_per_class[y][word] += 1

        
    def train(self, X, Y):
        
        ''' Train the multiclass (or Binary) Bayes Rule using the given 
            X [m x d] data matrix and Y labels matrix
            
            Input:
            ------
            X: [m x d] a data matrix of m d-dimensional examples.
            Y: [m x 1] a label vector.
            
            Returns:
            -----------
            Nothing
            '''
        # YOUR CODE HERE
        #get the length of total_documents in data
        self.doc_total = len(Y)
        # total number of documents coming in each class
        doc_per_class = defaultdict(int)


        for index, doc in enumerate(X):
            #get the current label
            current_label = Y[index]
            #increment the count of documents coming under the label extracted
            doc_per_class[current_label] += 1
            # Ensure that the document is a string, not a numpy array
            if isinstance(doc, np.ndarray):
                doc = ' '.join(doc)  # Convert numpy array to string
            elif isinstance(doc, bytes):
                doc = doc.decode('utf-8')  # Convert bytes to string

                
                  
            # for the given text string(doc) we will extract the tokens from it
            extracted_tokens = get_tokens(doc) 
            extracted_tokens = [lemmatizer.lemmatize(token) for token in extracted_tokens]
            extracted_tokens = [stemmer.stem(word) for word in extracted_tokens]
            # now we are updating the count of tokens coming under the current class label
            self.class_count[current_label] += len(extracted_tokens) 
            #update the overall vocabulary now
            self.vocabulary.update(extracted_tokens)
            # now we will update the count of the token coming in either positive/negative class
            for word in extracted_tokens:
                #ju label extract keya ha osi ka word count increase karna ha
                self.wordcount_per_class[current_label][word]+=1

        # now calculating the prior probabilites of the class 'pos' and 'neg'
        for cls, count in doc_per_class.items():
            # saved the priors in the dict
            self.prior_probs[cls] = count / self.doc_total


    def compute_Likelihood(self,token,cls):
        #cls : label, token: word
        # we did plus one for Laplace soothing
        eps = 3
        if token not in self.vocabulary:
            # If it doesn't exist, add the token to the vocabulary
            self.vocabulary.add(token)
            # Initialize the token count for the class
            self.wordcount_per_class[cls][token] = 0
        #like just in case if we didn't get the word in the vocab
        token_occurence_count = self.wordcount_per_class[cls][token] + eps
        total_tokens_in_cls = self.class_count[cls] + len(self.vocabulary) + 3
        #total_tokens_in_cls = len(self.vocabulary) 
        #total_tokens_in_cls = self.class_count[cls]
        likelihood = token_occurence_count / total_tokens_in_cls
        return likelihood


    def test(self, X):
        
        ''' Test the trained classifiers on the given set of examples 
        
                   
            Input:
            ------
            X: [m x d] a data matrix of m d-dimensional test examples.
           
            Returns:
            -----------
                pclass: the predicted class for each example, i.e. to which it belongs
        '''

        # her test k leyay we will call predict
        # X = np.array(X)
        # nexamples, nfeatures=X.shape
        nexamples = len(X)
        # YOUR CODE HERE
        preds = [] #this will store the resultant labels (which we will predict)
        for example in X:
            # extracting the example from X_test data
            # then sending that tokens in the example in the predict class
            class_pred = self.predict(example)
            # then predict function will return the predicted class of those tokens
            # predict wala function is basically applying the Naive Bayes Rules
            # or phir yeh prediction hum list mein append karatay jaien gay
            preds.append(class_pred)
        return np.array(preds)
        
    def predict(self, x):
        '''
            Predict the label of given input example...
            
            Input
            ---------
            x: example (list of words)
            
        '''
        # YOUR CODE HERE
        # Ensure input is a string
        if isinstance(x, np.ndarray):
            x = ' '.join(x)  # Convert numpy array to string
        elif isinstance(x, bytes):
            x = x.decode('utf-8')  # Convert bytes to string
        elif isinstance(x, list):
            x = x[0]  # Take the first element if it's a list
        extracted_tokens = get_tokens(x)
        extracted_tokens = [lemmatizer.lemmatize(token) for token in extracted_tokens]
        extracted_tokens = extracted_tokens = [stemmer.stem(word) for word in extracted_tokens]
        
        epsilon = 0.23
        # for each label this will contain the final probabilities
        extracted_n_computed_probs = defaultdict(float)
        for cls in self.prior_probs:
            #for each class extract the probability
            # get the prior prob first
            extracted_n_computed_probs[cls] = np.log(self.prior_probs[cls])
            #extracted_n_computed_probs[cls] = self.prior_probs[cls]
            #now theprobabilites of each token in that x
            for tok in extracted_tokens:
                #get the likelihood
                LikeLihood = self.compute_Likelihood(tok,cls)
                #add the likelihood in the prior probs
                #extracted_n_computed_probs[cls] += LikeLihood
                extracted_n_computed_probs[cls] = extracted_n_computed_probs[cls] + np.log(LikeLihood)
    
        #returning the class with highest class probs
        value_function = extracted_n_computed_probs.get
        max_prob_class = max(extracted_n_computed_probs, key=value_function)

        # Step 2: Get the maximum probability using the class
        max_prob = extracted_n_computed_probs[max_prob_class]
        max_prob = np.exp(max_prob)
        return max_prob_class
                
                


            




### Testing the implemented Naive Bayes Classifier

In [29]:
nb=NaiveBayes(classes)
nb.train(traindata,trainlabels)
pclasses=nb.test(testdata)
acc=np.sum(pclasses==testlabels)/float(testlabels.shape[0])

In [30]:
print(f'Accuracy of the Implemented Naive Classifier: {acc}') 

Accuracy of the Implemented Naive Classifier: 0.8025


### Test Cells Start
#### Do not Modify

In [31]:
from nose.tools import assert_in
nb=NaiveBayes(classes)
nb.train(traindata,trainlabels)
assert_equal (nb.test(testdata).shape[0], testdata.shape[0])
assert_in(type(nb.predict(["ok"])), [str, np.string_, np.str_], "Predict should return a label not list or array")

In [32]:
from nose.tools import assert_greater
nb=NaiveBayes(classes)
nb.train(traindata,trainlabels)
pclasses=nb.test(testdata)
acc=np.sum(pclasses==testlabels)/float(testlabels.shape[0])
assert_greater(acc, 0.77, "Acc must be greater then 77% you are doing something wrong")   